In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!python --version
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
^C
ERROR: Operation cancelled by user


ModuleNotFoundError: No module named 'nvcc4jupyter'

In [ ]:
!nvcc --version

In [ ]:
!g++ -v

In [ ]:
%%writefile cpphw.hpp

#include <iostream>

void printHelloWorld()

{

std::cout << "HelloWorld\nHelloWorld";

}

In [ ]:
%%writefile cpphw.cpp

#include "cpphw.hpp"

int main()

{

printHelloWorld();

return 0;

}

In [ ]:
ls

In [ ]:
%%script bash
g++ cpphw.cpp -std=c++11 -o hw.out
./hw.out

# Day 1 (16 Feb, 2025)

In [ ]:
%%writefile vect_add_cpu.cpp
// Sequential Vector Addition in CPP

#include <iostream>
#include <vector>
#include <ctime>
using namespace std;

int main(){
    clock_t start = clock();
    int n = 200000000;
    vector<int> A(n), B(n), C(n);
    for (int i=0; i<n; i++){
        A[i] = i+1;
        B[i] = (i+1)*2;
    }

    for (int i=0;i<n; i++){
        C[i] = A[i] + B[i];

    }

    clock_t end = clock();

    double duration = static_cast<double>(end - start) / CLOCKS_PER_SEC; // Time in seconds

    std::cout << "Time taken: " << duration * 1000 << " milliseconds" << std::endl; // Convert to milliseconds

    for (int i=0;i <20; i++){
        cout << C[i] << " ";
    }
    return 0;

}

In [ ]:
%%script bash
g++ vect_add_cpu.cpp -std=c++11 -o cpu.out
./cpu.out

In [ ]:
%%writefile vect_add_gpu.cu
// vector addition on gpu

#include <iostream>
#include <vector>
#include <ctime>
#include <cuda_runtime.h>

using namespace std;

__global__ void vector_addition(int *A, int *B, int *C, int n){
    // std::cout << "blockIdx.x = " << blockIdx.x << endl ;
    // std::cout << "blockDim.x = " << blockDim.x << endl ;
    // std::cout << "threadIdx.x = " << threadIdx.x << endl ;

    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if (i<n){
        C[i] = A[i] + B[i];
    }
    
};

int main(){
    int n = 200000000;
    vector<int> A(n), B(n), C(n);
    for (int i=0; i<n; i++){
        A[i] = i+1;
        B[i] = (i+1)*2;
    }

    clock_t start = clock();
    int *d_A, *d_B, *d_C;
    cudaMalloc((void **)&d_A, n*sizeof(int));
    cudaMalloc((void **)&d_B, n*sizeof(int));
    cudaMalloc((void **)&d_C, n*sizeof(int));

    cudaMemcpy(d_A, A.data(), n*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B.data(), n*sizeof(int), cudaMemcpyHostToDevice);

    int blockSize = 2;
    int gridSize = n+blockSize - 1;
    
    vector_addition <<<gridSize, blockSize>>>(d_A, d_B, d_C, n);
    cudaMemcpy(C.data(), d_C, n*sizeof(int), cudaMemcpyDeviceToHost);

    clock_t end = clock();

    double duration = static_cast<double>(end - start) / CLOCKS_PER_SEC; // Time in seconds

    std::cout << "Time taken: " << duration * 1000 << " milliseconds" << std::endl; // Convert to milliseconds

    for (int i=0;i <20; i++){
        cout << C[i] << " ";
    }
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    
    return 0;  
}



In [ ]:
%%script bash
nvcc vect_add_gpu.cu -o kernel
./kernel

## Notes:
* Gird Dimension: gridDim.x, gridDim.y, gridDim.z is the number of blocks in each direction of Grid.
* Block Dimension: blockDim.x, blockDim.y, blockDim.z, is the number of threads in each direction of block.
* Block Index: blockIdx.x[y,z], index of the block within the grid
* Thread Index: threadIdx.x[y,z], index of thread within a block
  
threadId =

            blockDim.x\*blockIdx.x + blockDim.x\*threadIdx.y + threadIdx.x (simple 2D within a block) +
              
            gridDim.x\*blockDim.y\*blockIdx.y (blocks of threads in previous row) 


**Steps:**
1. Define the function to run on GPU.
2. Declare variables on CPU and allocate necessary memory on GPU
3. Copy the data from CPU to GPU on the allocated locations.
4. Call the kernel
5. Save the copy of the task result on the CPU.
6. free the memory on GPU


# Day 2 (17 Feb, 2025)

In [ ]:
%%writefile vect_2D_add_gpu.cu
// vector addition on gpu

#include <iostream>
#include <vector>
#include <ctime>
#include <cuda_runtime.h>

using namespace std;

__global__ void vector_addition(int *A, int *B, int *C, int rows, int cols){
    // std::cout << "blockIdx.x = " << blockIdx.x << endl ;
    // std::cout << "blockDim.x = " << blockDim.x << endl ;
    // std::cout << "threadIdx.x = " << threadIdx.x << endl ;

    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if (row < rows && col < cols){
        int index = row*cols + col;
       // std::cout << index << endl;
        C[index] = A[index] + B[index];
    }
    
};

void print_vector(vector<vector <int>> matrix){
  // vector<vector<int>> matrix(3);
    for (vector vec: matrix){
      for (int val : vec){
        cout << val << " ";
      }
      cout << endl;
    }
}

// Helper function to check for CUDA errors
void checkCudaError(cudaError_t error) {
    if (error != cudaSuccess) {
        std::cerr << "CUDA Error: " << cudaGetErrorString(error) << std::endl;
        exit(EXIT_FAILURE);
    }
}

int main(){
    
    int rows = 30000;
    int cols = 30000;
    int n = rows*cols;
    vector<int> h_A(n), h_B(n), h_C(n);
    for (int i = 0; i < rows; ++i) {
        for (int j = 0; j < cols; ++j) {
            h_A[i * cols + j] = 2; // Example initialization for A
            h_B[i * cols + j] = -1;
                }
    }

    clock_t start_cpu = clock();
    for (int i = 0; i < rows; ++i) {
        for (int j = 0; j < cols; ++j) {
            h_C[i * cols + j] = h_A[i * cols + j] + h_B[i * cols + j];
                }
    }
    clock_t end_cpu = clock();
    double duration_cpu = static_cast<double>(end_cpu - start_cpu) / CLOCKS_PER_SEC; // Time in seconds
    std::cout << "CPU Time taken: " << duration_cpu * 1000 << " milliseconds" << std::endl; 

    clock_t start = clock();
    int *d_A, *d_B, *d_C;
    checkCudaError(cudaMalloc((void **)&d_A, n*sizeof(int)));
    checkCudaError(cudaMalloc((void **)&d_B, n*sizeof(int)));
    checkCudaError(cudaMalloc((void **)&d_C, n*sizeof(int)));

    checkCudaError(cudaMemcpy(d_A, h_A.data(), n*sizeof(int), cudaMemcpyHostToDevice));
    checkCudaError(cudaMemcpy(d_B, h_B.data(), n*sizeof(int), cudaMemcpyHostToDevice));

    // dim3 blockSize(rows, cols); // Block dimension (threads per block in x and y) - adjust based on matrix size and GPU capabilities
    // dim3 gridSize((cols + blockSize.x - 1) / blockSize.x, (rows + blockSize.y - 1) / blockSize.y);

    dim3 blockSize(16, 16); // Example block size
    blockSize.x = min(blockSize.x, cols); // Clamp block size to cols
    blockSize.y = min(blockSize.y, rows); // Clamp block size to rows
    dim3 gridSize((cols + blockSize.x - 1) / blockSize.x, (rows + blockSize.y - 1) / blockSize.y);
    
    vector_addition <<<gridSize, blockSize>>>(d_A, d_B, d_C, rows, cols);
    checkCudaError(cudaGetLastError());
    clock_t d2h_start = clock();
    checkCudaError(cudaMemcpy(h_C.data(), d_C, n*sizeof(int), cudaMemcpyDeviceToHost));
    clock_t d2h_end = clock();
    double d2h_duration = static_cast<double>(d2h_end - d2h_start) / CLOCKS_PER_SEC; // Time in seconds

    std::cout << "MemCpy D2H Time taken: " << d2h_duration * 1000 << " milliseconds" << std::endl; 

    clock_t end = clock();

    double duration_cuda = static_cast<double>(end - start) / CLOCKS_PER_SEC; // Time in seconds

    std::cout << "CUDA Time taken: " << duration_cuda * 1000 << " milliseconds" << std::endl; 
    // Convert to milliseconds

    cout << "CUDA Matrix Addition (2D):" << endl;
    
    for (int i = 0; i < 10; ++i) {
        for (int j = 0; j < 10; ++j) {
            cout << h_C[i * cols + j] << " ";
        }
        cout << endl;
    }
    checkCudaError(cudaFree(d_A));
    checkCudaError(cudaFree(d_B));
    checkCudaError(cudaFree(d_C));
    checkCudaError(cudaDeviceSynchronize());
    
    return 0;
}

In [ ]:
%%script bash
nvcc vect_2D_add_gpu.cu -o vect_2d
./vect_2d

# Day 3 (18 Feb, 2025)

Chapter 3 Notes:
* dim3 dimGrid(32,1,1), dim dimBlock(128,1,1) or dim3 dog(32,1,1), dim3 cat(128,1,1)
* vec_add <<<dimGrid or dog, dimBlock or cat>>> // we can use both with dim3 type, it is vector of int type with 3 elements, x, y, z.
* "For convenience, CUDA provides a special shortcut for calling a kernel with one-dimensional (1D) grids and blocks. Instead of using dim3 variables, one can use arithmetic expressions to specify the configuration of 1D grids and blocks. In this case, the CUDA compiler simply takes the arithmetic expression as the x dimensions and assumes that the y and z dimensions are 1." Page 49
* "the gridDim and blockDim are built-in variables in a kernel and always reflect the dimensions of the grid and the blocks, respectively."
* the allowed values of gridDim.x range from 1 to 231 2 1,1 and those of gridDim.y and gridDim.z range from 1 to 216 2 1 (65,535).
* The total size of a block in current CUDA systems is limited to 1024 threads. These threads can be distributed across the three dimensions in any way as long as the total number of threads does not exceed 1024.
* We need to know the no of cols at the compile time to accept dynamically allocated  arrays, but we do not have this info, as a result we flattne the dynamically allocated 2D arrays into an equivalent 1D arrays.
* The linearized access to a 3D array P will be in the form of P[plane * m * n +row * m + col].

In [ ]:
%%writefile matrix_mul.cu

#include <iostream>
#include <vector>
#include <ctime>
#include <cuda_runtime.h>

using namespace std;

__global__ void mat_mul(int *P_A, int *P_B,int *P_C, int Width){
    int row = blockDim.y*blockIdx.y + threadIdx.y;
    int col = blockDim.x*blockIdx.x + threadIdx.x;

    if (row < Width && col < Width){
        for (int k=0; k<Width; k++){
            P_C[row*Width+col] += P_A[row*Width+k] * P_B[k*Width+col];
        }
    }
}

void checkCudaError(cudaError_t error) {
    if (error != cudaSuccess) {
        std::cerr << "CUDA Error: " << cudaGetErrorString(error) << std::endl;
        exit(EXIT_FAILURE);
    }
}
int main(){
    int rows = 30000;
    int cols = 30000;
    int n = rows*cols;
    vector<int> h_A(n), h_B(n), h_C(n);
    for (int i = 0; i < rows; ++i) {
        for (int j = 0; j < cols; ++j) {
            h_A[i * cols + j] = 1; // Example initialization for A
            h_B[i * cols + j] = 2;
                }
    }
    clock_t start = clock();
    int *d_A, *d_B, *d_C;
    checkCudaError(cudaMalloc((void **)&d_A, n*sizeof(int)));
    checkCudaError(cudaMalloc((void **)&d_B, n*sizeof(int)));
    checkCudaError(cudaMalloc((void **)&d_C, n*sizeof(int)));

    checkCudaError(cudaMemcpy(d_A, h_A.data(), n*sizeof(int), cudaMemcpyHostToDevice));
    checkCudaError(cudaMemcpy(d_B, h_B.data(), n*sizeof(int), cudaMemcpyHostToDevice));

    int width = rows;
    dim3 blockSize(32, 32); // Example block size
    blockSize.x = min(blockSize.x, cols); // Clamp block size to cols
    blockSize.y = min(blockSize.y, rows); // Clamp block size to rows
    dim3 gridSize((cols + blockSize.x - 1) / blockSize.x, (rows + blockSize.y - 1) / blockSize.y);

    clock_t gpu_start = clock();
    mat_mul <<<gridSize, blockSize>>>(d_A, d_B, d_C, width);
    clock_t gpu_end = clock();
    double gpu_duration = static_cast<double>(gpu_end - gpu_start) / CLOCKS_PER_SEC; // Time in seconds
    std::cout << "GPU Time taken: " << gpu_duration * 1000 << " milliseconds" << std::endl; 
    
    clock_t d2h_start = clock();
    checkCudaError(cudaMemcpy(h_C.data(), d_C, n*sizeof(int), cudaMemcpyDeviceToHost));
    clock_t d2h_end = clock();
    double d2h_duration = static_cast<double>(d2h_end - d2h_start) / CLOCKS_PER_SEC; // Time in seconds
    std::cout << "MemCpy D2H Time taken: " << d2h_duration * 1000 << " milliseconds" << std::endl; 

    checkCudaError(cudaGetLastError());
    cout << "CUDA Matrix Mul (2D):" << endl;
    
    for (int i = 0; i < 10; ++i) {
        for (int j = 0; j < 10; ++j) {
            cout << h_C[i * cols + j] << " ";
        }
        cout << endl;
    }
    return 0;
}

In [ ]:
%%script bash
nvcc matrix_mul.cu -o mul
./mul

In [ ]:
!nvcc --version

# Day 4 (19 Feb, 2025)

### Notes
#### Shared Memory
* On-chip memory that is physically located close to the GPU cores, unlike global memory (DRAM) which is off-chip.
* Low Latency, High bandwith, Shared within a block, Limited Size.

In [ ]:
!python --version
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

In [ ]:
%%writefile red.cu
#include <iostream>
#include <vector>
#include <cuda_runtime.h>
#include <ctime>

using namespace std;
void checkCudaError(cudaError_t error){
    if (error != cudaSuccess){
        cerr << "Cuda Error: " << cudaGetErrorString(error) << endl;
        exit(EXIT_FAILURE);
    }
}

__global__ void reduction_tree(float *input, float*output, int block_size){

    __shared__ float shared_data[256];

    int thread_id = threadIdx.x;
    int block_id = blockIdx.x;
    shared_data[thread_id] = input[block_id*block_size + thread_id];
    __syncthreads();

    for (int stride=block_size / 2; stride > 0; stride/=2){
        if (thread_id < stride){
            shared_data[thread_id] += shared_data[thread_id + stride];
        }
        __syncthreads();
    }
    if (thread_id == 0){
        output[block_id] = shared_data[0];
    }
}

int main(){
    int array_size = 1024*1024*4*4;
    int block_size = 32;

    vector<float> h_input(array_size);
    vector<float> h_output((array_size + block_size - 1) / block_size, 0.0f);

    for (int i=0; i < array_size; i++){
        // h_input[i] = (float) (i%10+1);
        h_input[i] = 1;
    }
    
    float *d_input, *d_output;
    checkCudaError(cudaMalloc((void**)&d_input, array_size * sizeof(float)));
    checkCudaError(cudaMalloc((void**)&d_output, h_output.size() * sizeof(float)));

    checkCudaError(cudaMemcpy(d_input, h_input.data(), array_size * sizeof(float), cudaMemcpyHostToDevice));

    dim3 blockDim(block_size);
    dim3 gridDim((array_size+block_size-1)/block_size);

    clock_t cpu_start = clock();
    int array_sum = 0;
    for (int i=0; i < array_size; i++){
        array_sum += h_input[i];
    }
    clock_t cpu_end = clock();
    double cpu_duration = static_cast<double>(cpu_end - cpu_start) / CLOCKS_PER_SEC; // Time in seconds
    std::cout << "CPU Time taken: " << cpu_duration * 1000 << " milliseconds" << std::endl; 

    clock_t gpu_start = clock();
    reduction_tree <<<gridDim, blockDim>>>(d_input, d_output, block_size);
    clock_t gpu_end = clock();
    double gpu_duration = static_cast<double>(gpu_end - gpu_start) / CLOCKS_PER_SEC; // Time in seconds
    std::cout << "GPU Time taken: " << gpu_duration * 1000 << " milliseconds" << std::endl; 
    checkCudaError(cudaMemcpy(h_output.data(), d_output, h_output.size() * sizeof(float), cudaMemcpyDeviceToHost));

    int gpu_sum = 0;
    for (int i = 0; i < h_output.size(); i++){
        gpu_sum += h_output[i];
    }
    cout << "shared memory array size: " << 256 << endl;
    cout << "gpu sum " << gpu_sum<< endl;
    cout << "cpu sum " << array_sum << endl;
    checkCudaError(cudaFree(d_input));
    checkCudaError(cudaFree(d_output));
    
}

In [ ]:
%%script bash
nvcc red.cu -o red
./red

In [ ]:
{32: 0.225 milliseconds,
 64: 0.219,
 128: 0.182,
 256: 0.212,
 512:
}

# Day 5 (Feb 20, 2025)
* with blockSize > 256 when shared memory is 256, we are likely writing out of bounds of the sharedData array, leading to undefined behavior and potential errors.
* dynamic shared memory, you declare it as extern __shared__ float sharedData[]; in the kernel and then specify the size in bytes when you launch the kernel using the sharedMemConfig parameter of the kernel launch <<<>>>.

In [ ]:
    for (int i=0; i < neurons; i++ ){
        for (int j=0; j < sample; j++){
            cout << i << j << " " ;
            cout <<  neurons*i + j << endl;
            
            h_input[neurons*i + j] = neurons*i + j;
        }
    }


In [ ]:
%%writefile navie_layer_norm.cpp
#include <iostream>
#include <vector>
#include <ctime>
#include <stdio.h>
#include <cmath>

using namespace std;
void print_matrix(vector<float> matrix, int neurons, int sample ){
    for (int i=0; i < neurons; i++ )
        {
        for (int j=0; j < sample; j++)
        {
            cout <<  matrix[sample*i + j] << " ";
        }
        cout << endl ;
        }
    }
int main()
    {
    //ex1 - a11, a21, a31; ex2 - a12, a22, a32, mean1 = (a11 + a12)/2
    
    int sample = 100000;  // cols
    int neurons = 32; // rows

    int array_size = sample * neurons;
    vector<float> h_input(array_size);
    
    for (int i=0; i < neurons; i++ ){
        for (int j=0; j < sample; j++){
            h_input[sample*i + j] = sample*i + j;
        }
    }

    // printf("input matrix \n");
    // print_matrix(h_input, neurons, sample);

    vector<float> h_mean(sample);
    vector<float> h_var(sample, 0);
    vector<float> h_std(sample, 0);
    
    for (int i=0; i < neurons; i++ ){
        float col_sum = 0;
        for (int j=0; j < sample; j++){
            h_mean[j] += h_input[sample*i + j]; 
            }
        }
    for (int j=0; j < sample; j++){
         h_mean[j] = h_mean[j]/neurons;
        }
//    printf("col mean: \n");
//    for (float val: h_mean){
//        cout << (val) << " ";
//    }
//    printf("\n");

    for (int i=0; i < neurons; i++ ){
        for (int j=0; j < sample; j++){ 
            h_var[j] += ( h_input[sample*i + j] - h_mean[j]) * ( h_input[sample*i + j] - h_mean[j]); 
            }
        }

    for (int j=0; j < sample; j++){
         h_std[j] = sqrt(h_var[j]/neurons);
        }

//   printf("col diff sqr sum: \n");
//   for (float val: h_var){
//       cout << (val) << " ";
//   }
//   printf("\n");
//
//   printf("col std: \n");
//   for (float val: h_std){
//       cout << (val) << " ";
//   }
//   printf("\n\nprinting output matrix \n\n");
//
    clock_t cpu_start = clock();
    vector<float> h_output(array_size,0);
    for (int i=0; i < neurons; i++ ){
        for (int j=0; j < sample; j++){
            h_output[sample*i + j] = (h_input[sample*i + j] - h_mean[j])/(h_std[j] + 1e-7);
            float val =  h_output[sample*i + j];
            // cout << (val) << " ";
        }
        // cout << endl;
    }
   
    
    for (int i=0; i < neurons; i++ ){
        for (int j=0; j < sample; j++){
            h_output[sample*i + j] = (h_input[sample*i + j] - h_mean[j])/(h_std[j] + 1e-7);
            float val =  h_output[sample*i + j];
            // cout << (val) << " ";
        }
        // cout << endl;
    }
    clock_t cpu_end = clock();
    double cpu_duration = static_cast<double>(cpu_end - cpu_start) / CLOCKS_PER_SEC; // Time in seconds
    std::cout << "CPU Time taken: " << cpu_duration * 1000 << " milliseconds" << std::endl; 

    return 0;
    }


In [ ]:
%%script bash
g++ navie_layer_norm.cpp -o layer
./layer

In [ ]:
!python --version
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

# Day 6 (Feb 21, 2025)

In [ ]:
%%writefile layerNorm1.cu
#include <vector>
#include <iostream>
#include <ctime>
#include <cuda_runtime.h>
#include <cmath>

using namespace std;

__global__ void layer_norm(float *data, float *mean, float *var, float *output,  int neurons, int sample){
    int idx = blockIdx.x *  blockDim.x + threadIdx.x;
   // std::cout << idx;
    if (idx  <  gridDim.x * blockDim.x ){
        mean[idx] = 0.0f;
        var[idx] = 0.0f;
        for (int i=0; i < neurons; i++ ){

            int row_idx = idx*neurons + i;
            mean[idx] += data[row_idx];
            
        }
        mean[idx] /= neurons;
        for (int i=0; i < neurons; i++ ){

            int row_idx = idx*neurons + i;
            var[idx] += ((data[row_idx]-mean[idx]) * (data[row_idx]-mean[idx]));
            
        }
        var[idx] /= neurons;
        for (int i=0; i < neurons; i++ ){

            int row_idx = idx*neurons + i;
            output[row_idx] = (data[row_idx] - mean[idx])/(sqrt(var[idx])+1e-7);
            
        }
        }
}

int main(){
    int sample = 10024;
    int neurons = 10024;
    vector<float> h_data(sample*neurons);
    vector<float> h_out(sample*neurons);
    vector<float> h_mean(sample);
    vector<float> h_var(sample); 

    for (int i =0; i < sample; i++){
        for (int j =0; j < neurons; j++){
        h_data[i*neurons + j] = i*neurons + j;
        }
    }

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    float *d_data, *d_out, *d_mean, *d_var;
    cudaMalloc((void **)&d_data, sample*neurons*sizeof(float));
    cudaMalloc((void **)&d_out, sample*neurons*sizeof(float));
    cudaMalloc((void **)&d_mean, sample*sizeof(float));
    cudaMalloc((void **)&d_var, sample*sizeof(float));

    cudaMemcpy(d_data, h_data.data(), sample*neurons*sizeof(float), cudaMemcpyHostToDevice );
    cudaMemcpy(d_out, h_out.data(), sample*neurons*sizeof(float), cudaMemcpyHostToDevice );
    cudaMemcpy(d_mean, h_mean.data(), sample*sizeof(float), cudaMemcpyHostToDevice );
    cudaMemcpy(d_var, h_var.data(), sample*sizeof(float), cudaMemcpyHostToDevice );

    dim3 blockSize(256);
    dim3 gridSize((sample*neurons + blockSize.x - 1) / blockSize.x);  // Proper grid size calculation

    float total_ms = 0;
    for (int i = 0; i < 10; i++){
        cudaEventRecord(start);
        layer_norm<<<gridSize, blockSize>>>(d_data, d_mean, d_var, d_out, neurons, sample);
        cudaDeviceSynchronize();
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        float milliseconds = 0;
        cudaEventElapsedTime(&milliseconds, start, stop);
        total_ms += milliseconds;
    }
    float averageMilliseconds = total_ms / 10;
    std::cout << "Average kernel execution time: " << averageMilliseconds << " ms" << std::endl;

    cudaMemcpy(h_mean.data(), d_mean, sample*sizeof(float), cudaMemcpyDeviceToHost );
    cudaMemcpy(h_var.data(), d_var, sample*sizeof(float), cudaMemcpyDeviceToHost );
    cudaMemcpy(h_out.data(), d_out, sample*neurons*sizeof(float), cudaMemcpyDeviceToHost );

    cudaFree(d_out);
    cudaFree(d_mean);
    cudaFree(d_var);
    cudaFree(d_data);
    return 0;

    }


In [ ]:
%%script bash
nvcc layerNorm1.cu -o red1
./red1

In [ ]:
%%writefile elapsed.cu
#include <iostream>
#include <cuda_runtime.h>

// Simple kernel to perform vector addition
__global__ void vectorAdd(float* a, float* b, float* c, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        c[i] = a[i] + b[i];
    }
}

int main() {
    int n = 1024 * 1024; // Example size
    float* h_a, * h_b, * h_c; // Host arrays
    float* d_a, * d_b, * d_c; // Device arrays

    // Allocate memory on host
    h_a = new float[n];
    h_b = new float[n];
    h_c = new float[n];

    // Allocate memory on device
    cudaMalloc(&d_a, n * sizeof(float));
    cudaMalloc(&d_b, n * sizeof(float));
    cudaMalloc(&d_c, n * sizeof(float));

    // Initialize host arrays (example values)
    for (int i = 0; i < n; i++) {
        h_a[i] = i;
        h_b[i] = 2 * i;
    }

    // Copy data from host to device
    cudaMemcpy(d_a, h_a, n * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, n * sizeof(float), cudaMemcpyHostToDevice);


    // Kernel launch configuration
    int blockSize = 256;
    int gridSize = (n + blockSize - 1) / blockSize;

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    int numRuns = 100;
    float totalMilliseconds = 0;

        // Warm-up runs
    for (int i = 0; i < 5; ++i) {
        vectorAdd<<<gridSize, blockSize>>>(d_a, d_b, d_c, n);
        cudaDeviceSynchronize(); // Synchronize after warm-up runs
    }


    for (int i = 0; i < numRuns; ++i) {
        cudaEventRecord(start);
        vectorAdd<<<gridSize, blockSize>>>(d_a, d_b, d_c, n);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        float milliseconds = 0;
        cudaEventElapsedTime(&milliseconds, start, stop);
        totalMilliseconds += milliseconds;
    }

    float averageMilliseconds = totalMilliseconds / numRuns;
    std::cout << "Average kernel execution time: " << averageMilliseconds << " ms" << std::endl;

    // Copy results back to host (optional, for verification)
    //cudaMemcpy(h_c, d_c, n * sizeof(float), cudaMemcpyDeviceToHost);

    // Free memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    delete[] h_a;
    delete[] h_b;
    delete[] h_c;

    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaDeviceReset(); // Important: Release CUDA resources

    return 0;
}

In [ ]:
%%script bash
nvcc elapsed.cu -o elp
./elp

In [ ]:
!python --version
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

In [ ]:
%%writefile t.cu
#include <iostream>
#include <cuda_runtime.h>

const int TILE_DIM = 32;
const int num_runs = 100;

// Simple kernel to perform vector addition
__global__ void matrix_copy(float* in_matrix, float* out_matrix, int n) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int block_rows = blockDim.y;
    int size = blockDim.x * gridDim.x; // as rows == cols
    if (x < n && y < n)
        for (int dim_y = 0; dim_y < gridDim.y; dim_y += block_rows){
            out_matrix[(dim_y+ y)*size + x] = in_matrix[(dim_y+ y)*size + x];
        }
}

__global__ void navie_transpose(float* in_matrix, float* out_matrix, int n) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int block_rows = blockDim.y;
    int size = blockDim.x * gridDim.x; // as rows == cols
    if (x < n && y < n)
        for (int dim_y = 0; dim_y < gridDim.y; dim_y += block_rows){
            out_matrix[x*size + (dim_y + y)] = in_matrix[(dim_y + y)*size + x];
        }

}

__global__ void shared_copy(float* in_matrix, float* out_matrix, int n) {
    __shared__ float share[TILE_DIM * TILE_DIM];
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    // int block_rows = blockDim.y;
    // int size = blockDim.x * gridDim.x; // as rows == cols
    if (x < n && y < n)
        share[(threadIdx.y)*blockDim.y + threadIdx.x] = in_matrix[(y)*n + x];
        __syncthreads();
        // for (int dim_y = 0; dim_y < gridDim.y; dim_y += block_rows)
        {
        out_matrix[(y)*n + x] = share[(threadIdx.x)*blockDim.x + threadIdx.y];
        }

    }

__global__ void shared_coalesced_bank_conflict(float *in_matrix, float *out_matrix, int n){
    __shared__ float tile[TILE_DIM][TILE_DIM];
    int x = blockIdx.x*blockDim.x + threadIdx.x;
    int y = blockIdx.y*blockDim.y + threadIdx.y;

    int width = n;
    if (x < n && y < n){
        tile[threadIdx.y][threadIdx.x] = in_matrix[(y * width ) + threadIdx.x];
    }
    __syncthreads();
    
    x = blockIdx.y*blockDim.x + threadIdx.x;
    y = blockIdx.x*blockDim.y + threadIdx.y;
    if (x < n && y < n){
        out_matrix[(y*width) + x] = tile[threadIdx.x][threadIdx.y];
    }
 }

 __global__ void shared_coalesced_no_bank_conflict(float *in_matrix, float *out_matrix, int n){
    __shared__ float tile[TILE_DIM][TILE_DIM + 1];
    int x = blockIdx.x*blockDim.x + threadIdx.x;
    int y = blockIdx.y*blockDim.y + threadIdx.y;

    int width = n;
    if (x < n && y < n){
        tile[threadIdx.y][threadIdx.x] = in_matrix[(y * width ) + threadIdx.x];
    }
    __syncthreads();
    
    x = blockIdx.y*blockDim.x + threadIdx.x;
    y = blockIdx.x*blockDim.y + threadIdx.y;
    if (x < n && y < n){
        out_matrix[(y*width) + x] = tile[threadIdx.x][threadIdx.y];
    }
 }

int main() {
    int rows = 1024;
    int cols = 1024;
    int size = rows * cols; 
    float *c_data, *c_output; // Host arrays
    float *d_data, *d_output; // Device arrays

    // Allocate memory on host
    c_data = new float[size];
    c_output = new float[size];

    // Allocate memory on device
    cudaMalloc((void **)&d_data, size * sizeof(float));
    cudaMalloc((void **)&d_output, size * sizeof(float));
    // cudaMallocHost creates a pinned memory, avoids the overhead of pageable -> pinned memory
    // When executed cudaMallocHost, the execution time impacted, not all the time the pinned meory is efficient

    // Initialize host arrays (example values)
    for (int row = 0; row < rows; row ++){
        for (int col = 0; col < cols; col++){
            c_data[row*cols + col] = row*cols + col;
        }
    }

    // Copy data from host to device
    cudaMemcpy(d_data, c_data, size * sizeof(float), cudaMemcpyHostToDevice);


    // Kernel launch configuration
    dim3 blockSize(32, 32, 1);
    dim3 gridSize(32, 32, 1);

    printf("%25s%25s%25s\n", "Exectution", "Bandwidth (GB/s)", "GPU Time (ms)");
    printf("\n");

    printf("%25s","Matrix Copy");

    cudaEvent_t start1, stop1;
    cudaEventCreate(&start1);
    cudaEventCreate(&stop1);
    float total_milliseconds1 = 0;
    for (int i=0; i < num_runs; i++){
        cudaEventRecord(start1);
        matrix_copy<<<gridSize, blockSize>>>(d_data, d_output, rows);
        //navie_transpose<<<gridSize, blockSize>>>(d_data, d_output, rows);
        cudaEventRecord(stop1);
        cudaEventSynchronize(stop1);
        float milliseconds1 = 0;
        cudaEventElapsedTime(&milliseconds1, start1, stop1);
        total_milliseconds1 += milliseconds1;
    }
    float average_milliseconds1 = total_milliseconds1 / num_runs;
    cudaEventDestroy(start1);
    cudaEventDestroy(stop1);
    cudaMemcpy(c_output, d_output, size * sizeof(float), cudaMemcpyDeviceToHost);
    printf("%20.4f%25.4f\n", 2 * size * sizeof(float)* 1e-6 / average_milliseconds1, average_milliseconds1 );


    /*  -----------------------  */

    printf("%25s","Shared Copy");

    cudaEvent_t start2, stop2;
    cudaEventCreate(&start2);
    cudaEventCreate(&stop2);
    
    float total_milliseconds2 = 0;
    for (int i=0; i < num_runs; i++){
        cudaEventRecord(start2);
        shared_copy<<<gridSize, blockSize>>>(d_data, d_output, rows);
        cudaEventRecord(stop2);
        cudaEventSynchronize(stop2);
        float milliseconds2 = 0;
        cudaEventElapsedTime(&milliseconds2, start2, stop2);
        total_milliseconds2 += milliseconds2;
    }
    float average_milliseconds2 = total_milliseconds2 / num_runs;
    cudaEventDestroy(start2);
    cudaEventDestroy(stop2);
    cudaMemcpy(c_output, d_output, size * sizeof(float), cudaMemcpyDeviceToHost);
    printf("%20.4f%25.4f\n", 2 * size * sizeof(float)* 1e-6 / average_milliseconds2, average_milliseconds2 );


    /*  -----------------------  */

    printf("%25s","Navie Transpose");

    cudaEvent_t start3, stop3;
    cudaEventCreate(&start3);
    cudaEventCreate(&stop3);
    float total_milliseconds3 = 0;
    for (int i=0; i < num_runs; i++){
        cudaEventRecord(start3);
        navie_transpose<<<gridSize, blockSize>>>(d_data, d_output, rows);
        cudaEventRecord(stop3);
        cudaEventSynchronize(stop3);
        float milliseconds3 = 0;
        cudaEventElapsedTime(&milliseconds3, start3, stop3);
        total_milliseconds3 += milliseconds3;
    }
    float average_milliseconds3 = total_milliseconds3 / num_runs;
    cudaEventDestroy(start3);
    cudaEventDestroy(stop3);
    cudaMemcpy(c_output, d_output, size * sizeof(float), cudaMemcpyDeviceToHost);
    printf("%20.4f%25.4f\n", 2 * size * sizeof(float)* 1e-6 / average_milliseconds3, average_milliseconds3 );


    /*  -----------------------  */

    printf("%25s","Shared Coalesced");

    cudaEvent_t start4, stop4;
    cudaEventCreate(&start4);
    cudaEventCreate(&stop4);
    float total_milliseconds4 = 0;
    for (int i=0; i < num_runs; i++){
        cudaEventRecord(start4);
        shared_coalesced_bank_conflict<<<gridSize, blockSize>>>(d_data, d_output, rows);
        cudaEventRecord(stop4);
        cudaEventSynchronize(stop4);
        float milliseconds4 = 0;
        cudaEventElapsedTime(&milliseconds4, start4, stop4);
        total_milliseconds4 += milliseconds4;
    }
    float average_milliseconds4 = total_milliseconds4 / num_runs;
    cudaEventDestroy(start4);
    cudaEventDestroy(stop4);
    cudaMemcpy(c_output, d_output, size * sizeof(float), cudaMemcpyDeviceToHost);
    printf("%20.4f%25.4f\n", 2 * size * sizeof(float)* 1e-6 / average_milliseconds4, average_milliseconds4 );



    /*  -----------------------  */

    printf("%25s","No Bank Conflict");

    cudaEvent_t start5, stop5;
    cudaEventCreate(&start5);
    cudaEventCreate(&stop5);
    float total_milliseconds5 = 0;
    for (int i=0; i < num_runs; i++){
        cudaEventRecord(start5);
        shared_coalesced_no_bank_conflict<<<gridSize, blockSize>>>(d_data, d_output, rows);
        cudaEventRecord(stop5);
        cudaEventSynchronize(stop5);
        float milliseconds5 = 0;
        cudaEventElapsedTime(&milliseconds5, start5, stop5);
        total_milliseconds5 += milliseconds5;
    }
    float average_milliseconds5 = total_milliseconds5 / num_runs;
    cudaEventDestroy(start5);
    cudaEventDestroy(stop5);
    cudaMemcpy(c_output, d_output, size * sizeof(float), cudaMemcpyDeviceToHost);
    printf("%20.4f%25.4f\n", 2 * size * sizeof(float)* 1e-6 / average_milliseconds5, average_milliseconds5 );



    // for (int row = 0; row < 10; row ++){
    //     for (int col = 0; col < 10; col++){
    //         std::cout << c_data[row*cols + col] << " " ;
    //     }
    //     std::cout << "\n";
    // }

    // std::cout << "\n\n";

    // for (int row = 0; row < 10; row ++){
    //     for (int col = 0; col < 10; col++){
    //         std::cout << c_output[row*cols + col] << " " ;
    //     }
    //     std::cout << "\n";
    // }
    
    // Free memory
    cudaFree(d_data);
    cudaFree(d_output);
    delete[] c_data;
    delete[] c_output;


    cudaDeviceReset(); // Important: Release CUDA resources
    return 0;
}


In [ ]:
%%script bash
nvcc t.cu -o t
./t

# Pinned Memory

In [ ]:
%%writefile t1.cu
#include <iostream>
#include <cuda_runtime.h>

const int TILE_DIM = 32;
const int num_runs = 100;

// Simple kernel to perform vector addition
__global__ void matrix_copy(float* in_matrix, float* out_matrix, int n) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int block_rows = blockDim.y;
    int size = blockDim.x * gridDim.x; // as rows == cols
    if (x < n && y < n)
        for (int dim_y = 0; dim_y < gridDim.y; dim_y += block_rows){
            out_matrix[(dim_y+ y)*size + x] = in_matrix[(dim_y+ y)*size + x];
        }
}

__global__ void navie_transpose(float* in_matrix, float* out_matrix, int n) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int block_rows = blockDim.y;
    int size = blockDim.x * gridDim.x; // as rows == cols
    if (x < n && y < n)
        for (int dim_y = 0; dim_y < gridDim.y; dim_y += block_rows){
            out_matrix[x*size + (dim_y + y)] = in_matrix[(dim_y + y)*size + x];
        }

}

__global__ void shared_copy(float* in_matrix, float* out_matrix, int n) {
    __shared__ float share[TILE_DIM * TILE_DIM];
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    // int block_rows = blockDim.y;
    // int size = blockDim.x * gridDim.x; // as rows == cols
    if (x < n && y < n)
        share[(threadIdx.y)*blockDim.y + threadIdx.x] = in_matrix[(y)*n + x];
        __syncthreads();
        // for (int dim_y = 0; dim_y < gridDim.y; dim_y += block_rows)
        {
        out_matrix[(y)*n + x] = share[(threadIdx.x)*blockDim.x + threadIdx.y];
        }

    }

__global__ void shared_coalesced_bank_conflict(float *in_matrix, float *out_matrix, int n){
    __shared__ float tile[TILE_DIM][TILE_DIM];
    int x = blockIdx.x*blockDim.x + threadIdx.x;
    int y = blockIdx.y*blockDim.y + threadIdx.y;

    int width = n;
    if (x < n && y < n){
        tile[threadIdx.y][threadIdx.x] = in_matrix[(y * width ) + threadIdx.x];
    }
    __syncthreads();
    
    x = blockIdx.y*blockDim.x + threadIdx.x;
    y = blockIdx.x*blockDim.y + threadIdx.y;
    if (x < n && y < n){
        out_matrix[(y*width) + x] = tile[threadIdx.x][threadIdx.y];
    }
 }

 __global__ void shared_coalesced_no_bank_conflict(float *in_matrix, float *out_matrix, int n){
    __shared__ float tile[TILE_DIM][TILE_DIM + 1];
    int x = blockIdx.x*blockDim.x + threadIdx.x;
    int y = blockIdx.y*blockDim.y + threadIdx.y;

    int width = n;
    if (x < n && y < n){
        tile[threadIdx.y][threadIdx.x] = in_matrix[(y * width ) + threadIdx.x];
    }
    __syncthreads();
    
    x = blockIdx.y*blockDim.x + threadIdx.x;
    y = blockIdx.x*blockDim.y + threadIdx.y;
    if (x < n && y < n){
        out_matrix[(y*width) + x] = tile[threadIdx.x][threadIdx.y];
    }
 }

int main() {
    int rows = 1024;
    int cols = 1024;
    int size = rows * cols; 
    float *c_data, *c_output; // Host arrays
    float *d_data, *d_output; // Device arrays

    // Allocate memory on host
    c_data = new float[size];
    c_output = new float[size];

    // Allocate memory on device
    cudaMallocHost((void **)&d_data, size * sizeof(float));
    cudaMallocHost((void **)&d_output, size * sizeof(float));
    // cudaMallocHost creates a pinned memory, avoids the overhead of pageable -> pinned memory

    // Initialize host arrays (example values)
    for (int row = 0; row < rows; row ++){
        for (int col = 0; col < cols; col++){
            c_data[row*cols + col] = row*cols + col;
        }
    }

    // Copy data from host to device
    cudaMemcpy(d_data, c_data, size * sizeof(float), cudaMemcpyHostToDevice);


    // Kernel launch configuration
    dim3 blockSize(32, 32, 1);
    dim3 gridSize(32, 32, 1);

    printf("%25s%25s%25s\n", "Exectution", "Bandwidth (GB/s)", "GPU Time (ms)");
    printf("\n");

    printf("%25s","Matrix Copy");

    cudaEvent_t start1, stop1;
    cudaEventCreate(&start1);
    cudaEventCreate(&stop1);
    float total_milliseconds1 = 0;
    for (int i=0; i < num_runs; i++){
        cudaEventRecord(start1);
        matrix_copy<<<gridSize, blockSize>>>(d_data, d_output, rows);
        //navie_transpose<<<gridSize, blockSize>>>(d_data, d_output, rows);
        cudaEventRecord(stop1);
        cudaEventSynchronize(stop1);
        float milliseconds1 = 0;
        cudaEventElapsedTime(&milliseconds1, start1, stop1);
        total_milliseconds1 += milliseconds1;
    }
    float average_milliseconds1 = total_milliseconds1 / num_runs;
    cudaEventDestroy(start1);
    cudaEventDestroy(stop1);
    cudaMemcpy(c_output, d_output, size * sizeof(float), cudaMemcpyDeviceToHost);
    printf("%20.4f%25.4f\n", 2 * size * sizeof(float)* 1e-6 / average_milliseconds1, average_milliseconds1 );


    /*  -----------------------  */

    printf("%25s","Shared Copy");

    cudaEvent_t start2, stop2;
    cudaEventCreate(&start2);
    cudaEventCreate(&stop2);
    
    float total_milliseconds2 = 0;
    for (int i=0; i < num_runs; i++){
        cudaEventRecord(start2);
        shared_copy<<<gridSize, blockSize>>>(d_data, d_output, rows);
        cudaEventRecord(stop2);
        cudaEventSynchronize(stop2);
        float milliseconds2 = 0;
        cudaEventElapsedTime(&milliseconds2, start2, stop2);
        total_milliseconds2 += milliseconds2;
    }
    float average_milliseconds2 = total_milliseconds2 / num_runs;
    cudaEventDestroy(start2);
    cudaEventDestroy(stop2);
    cudaMemcpy(c_output, d_output, size * sizeof(float), cudaMemcpyDeviceToHost);
    printf("%20.4f%25.4f\n", 2 * size * sizeof(float)* 1e-6 / average_milliseconds2, average_milliseconds2 );


    /*  -----------------------  */

    printf("%25s","Navie Transpose");

    cudaEvent_t start3, stop3;
    cudaEventCreate(&start3);
    cudaEventCreate(&stop3);
    float total_milliseconds3 = 0;
    for (int i=0; i < num_runs; i++){
        cudaEventRecord(start3);
        navie_transpose<<<gridSize, blockSize>>>(d_data, d_output, rows);
        cudaEventRecord(stop3);
        cudaEventSynchronize(stop3);
        float milliseconds3 = 0;
        cudaEventElapsedTime(&milliseconds3, start3, stop3);
        total_milliseconds3 += milliseconds3;
    }
    float average_milliseconds3 = total_milliseconds3 / num_runs;
    cudaEventDestroy(start3);
    cudaEventDestroy(stop3);
    cudaMemcpy(c_output, d_output, size * sizeof(float), cudaMemcpyDeviceToHost);
    printf("%20.4f%25.4f\n", 2 * size * sizeof(float)* 1e-6 / average_milliseconds3, average_milliseconds3 );


    /*  -----------------------  */

    printf("%25s","Shared Coalesced");

    cudaEvent_t start4, stop4;
    cudaEventCreate(&start4);
    cudaEventCreate(&stop4);
    float total_milliseconds4 = 0;
    for (int i=0; i < num_runs; i++){
        cudaEventRecord(start4);
        shared_coalesced_bank_conflict<<<gridSize, blockSize>>>(d_data, d_output, rows);
        cudaEventRecord(stop4);
        cudaEventSynchronize(stop4);
        float milliseconds4 = 0;
        cudaEventElapsedTime(&milliseconds4, start4, stop4);
        total_milliseconds4 += milliseconds4;
    }
    float average_milliseconds4 = total_milliseconds4 / num_runs;
    cudaEventDestroy(start4);
    cudaEventDestroy(stop4);
    cudaMemcpy(c_output, d_output, size * sizeof(float), cudaMemcpyDeviceToHost);
    printf("%20.4f%25.4f\n", 2 * size * sizeof(float)* 1e-6 / average_milliseconds4, average_milliseconds4 );



    /*  -----------------------  */

    printf("%25s","No Bank Conflict");

    cudaEvent_t start5, stop5;
    cudaEventCreate(&start5);
    cudaEventCreate(&stop5);
    float total_milliseconds5 = 0;
    for (int i=0; i < num_runs; i++){
        cudaEventRecord(start5);
        shared_coalesced_no_bank_conflict<<<gridSize, blockSize>>>(d_data, d_output, rows);
        cudaEventRecord(stop5);
        cudaEventSynchronize(stop5);
        float milliseconds5 = 0;
        cudaEventElapsedTime(&milliseconds5, start5, stop5);
        total_milliseconds5 += milliseconds5;
    }
    float average_milliseconds5 = total_milliseconds5 / num_runs;
    cudaEventDestroy(start5);
    cudaEventDestroy(stop5);
    cudaMemcpy(c_output, d_output, size * sizeof(float), cudaMemcpyDeviceToHost);
    printf("%20.4f%25.4f\n", 2 * size * sizeof(float)* 1e-6 / average_milliseconds5, average_milliseconds5 );



    // for (int row = 0; row < 10; row ++){
    //     for (int col = 0; col < 10; col++){
    //         std::cout << c_data[row*cols + col] << " " ;
    //     }
    //     std::cout << "\n";
    // }

    // std::cout << "\n\n";

    // for (int row = 0; row < 10; row ++){
    //     for (int col = 0; col < 10; col++){
    //         std::cout << c_output[row*cols + col] << " " ;
    //     }
    //     std::cout << "\n";
    // }
    
    // Free memory
    cudaFree(d_data);
    cudaFree(d_output);
    delete[] c_data;
    delete[] c_output;


    cudaDeviceReset(); // Important: Release CUDA resources
    return 0;
}


In [ ]:
%%script bash
nvcc t1.cu -o t1
./t1

# Day XX , (March 2, 2025)

In [ ]:
1. define block size and grid size
    - block = (32, 32)
    - grid = (cols+blocksize-1/blocksize)
    - tile_width = 32


    bx, by
    tx, ty

    Mds, Nds

    Mds[ty][tx] = A[blockIdx.x * blockDim.x + tx]
    Nds[ty][tx]

    M[by*bdim*width + ty*width + tx ]
    N[bx*bdim + (ty*width) + tx]

    

In [ ]:
M = [[1,2,3], [4,5,6], [7,8,9]]
N = [[3,2,1], [6,5,4], [9,8,7]]

In [ ]:
%%writefile shared_mat_mul.cu

#include <iostream>
#include <cuda_runtime.h>

# define TILE_WIDTH 32

__global__ void navie_mat_mul(float *P_A, float *P_B,float *P_C, int Width){
    int row = blockDim.y*blockIdx.y + threadIdx.y;
    int col = blockDim.x*blockIdx.x + threadIdx.x;

    if (row < Width && col < Width){
        for (int k=0; k<Width; k++){
            P_C[row*Width+col] += P_A[row*Width+k] * P_B[k*Width+col];
        }
    }
}

__global__ void  mat_mul(float *A, float *B, float *C, int width){
    __shared__ float As[TILE_WIDTH][TILE_WIDTH];
    __shared__ float Bs[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int row = by * TILE_WIDTH + ty;
    int col = bx * TILE_WIDTH + tx;

    float element_value = 0;

    for (int t=0; t < int (width+TILE_WIDTH - 1)/TILE_WIDTH; t ++){

        if ((row < width) && ((t*TILE_WIDTH + tx) < width)){
            As[ty][tx] = A[row*width + t*TILE_WIDTH + tx];
        }
        else{
            As[ty][tx] = 0.0f;
        }
    
        if ((col < width) && ((t*TILE_WIDTH + ty) < width)){
            Bs[ty][tx] = B[(t*TILE_WIDTH + ty)*width + col];
        }
        else{
            Bs[ty][tx] = 0.0f;
        }
        __syncthreads();

        
        for (int k=0; k < TILE_WIDTH; k++){
            element_value += As[ty][k]* Bs[k][tx];
        }
        __syncthreads();

    }
    if ((row < width) && (col < width)){
        C[row*width + col] = element_value;
    }

}


int main(){
    int rows = 1024;
    int cols = 1024;
    int size = rows * cols; 
    float *hA, *hB, *hC, *hC_naive; // Host arrays
    float *dA, *dB, *dC, *dC_naive; // Device arrays

    // Allocate memory on host
    hA = new float[size];
    hB = new float[size];
    hC = new float[size];
    hC_naive = new float[size];

    // Allocate memory on device
    cudaMalloc((void **)&dA, size * sizeof(float));
    cudaMalloc((void **)&dB, size * sizeof(float));
    cudaMalloc((void **)&dC, size * sizeof(float));

    cudaMalloc((void **)&dC_naive, size * sizeof(float));

    // Initialize host arrays (example values)
    for (int row = 0; row < rows; row ++){
        for (int col = 0; col < cols; col++){
            // hA[row*cols + col] = row*cols + col;
            hA[row*cols + col] = 1;
        }
    }

    // Initialize host arrays (example values)
    for (int row = 0; row < rows; row ++){
        for (int col = 0; col < cols; col++){
            // hB[row*cols + col] = row*cols + col;
            hB[row*cols + col] = 1;
        }
    }

    // Copy data from host to device
    cudaMemcpy(dA, hA, size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(dB, hB, size * sizeof(float), cudaMemcpyHostToDevice);


    // Kernel launch configuration
    dim3 blockSize(TILE_WIDTH, TILE_WIDTH, 1);
    dim3 gridSize((cols + TILE_WIDTH - 1) / TILE_WIDTH, (rows + TILE_WIDTH - 1) / TILE_WIDTH, 1);

    printf("%25s%25s%25s\n", "Exectution", "Bandwidth (GB/s)", "GPU Time (ms)");
    printf("\n");

    printf("%25s","Matrix Mul");

    cudaEvent_t start1, stop1;
    cudaEventCreate(&start1);
    cudaEventCreate(&stop1);
    float total_milliseconds1 = 0;
    int num_runs = 100;

    for (int i=0; i < num_runs; i++){
        cudaEventRecord(start1);
        mat_mul<<<gridSize, blockSize>>>(dA, dB, dC, rows);
        cudaEventRecord(stop1);
        cudaEventSynchronize(stop1);
        float milliseconds1 = 0;
        cudaEventElapsedTime(&milliseconds1, start1, stop1);
        total_milliseconds1 += milliseconds1;
    }
    float average_milliseconds1 = total_milliseconds1 / num_runs;
    cudaEventDestroy(start1);
    cudaEventDestroy(stop1);
    cudaMemcpy(hC, dC, size * sizeof(float), cudaMemcpyDeviceToHost);
    printf("%20.4f%25.4f\n", 2 * size * sizeof(float)* 1e-6 / average_milliseconds1, average_milliseconds1 );


    
    printf("%25s","Navie Matrix Mul");

    cudaEvent_t start2, stop2;
    cudaEventCreate(&start2);
    cudaEventCreate(&stop2);
    float total_milliseconds2 = 0;
    int num_runs1 = 100;

    for (int i=0; i < num_runs1; i++){
        cudaEventRecord(start2);
        navie_mat_mul<<<gridSize, blockSize>>>(dA, dB, dC_naive, rows);
        cudaEventRecord(stop2);
        cudaEventSynchronize(stop2);
        float milliseconds2 = 0;
        cudaEventElapsedTime(&milliseconds2, start2, stop2);
        total_milliseconds2 += milliseconds2;
    }
    float average_milliseconds2 = total_milliseconds2 / num_runs1;
    cudaEventDestroy(start2);
    cudaEventDestroy(stop2);
    cudaMemcpy(hC_naive, dC_naive, size * sizeof(float), cudaMemcpyDeviceToHost);

    printf("%20.4f%25.4f\n", 2 * size * sizeof(float)* 1e-6 / average_milliseconds2, average_milliseconds2 );

    
    printf("hA: \n ");
    for (int row = 0; row < 5; row ++){
        for (int col = 0; col < 5; col++){
            printf("%0.0f ", hA[row*cols + col]);
        }
        printf("\n");
    }
    
    printf("hB: \n");
    for (int row = 0; row < 5; row ++){
        for (int col = 0; col < 5; col++){
            printf("%0.0f ", hB[row*cols + col]);
        }
        printf("\n");
    }

    printf("\n\n");
    printf("hC: \n");
    for (int row = 0; row < 5; row ++){
        for (int col = 0; col < 5; col++){
            printf("%0.f ", hC[row*cols + col]);
        }
        printf("\n");
    }

    printf("\n\n");
    printf("hC_naive: \n");
    for (int row = 0; row < 5; row ++){
        for (int col = 0; col < 5; col++){
            printf("%0.f ", hC_naive[row*cols + col]/num_runs1);
        }
        printf("\n");
    }
    return 0;

}

In [ ]:
%%script bash
nvcc ./shared_mat_mul.cu -o t
./t

In [ ]:
%%writefile mul.cu
#include <iostream>
#include <cuda_runtime.h>

# define TILE_WIDTH 32

__global__ void navie_mat_mul(float *P_A, float *P_B,float *P_C, int Width){
    int row = blockDim.y*blockIdx.y + threadIdx.y;
    int col = blockDim.x*blockIdx.x + threadIdx.x;

    if (row < Width && col < Width){
        for (int k=0; k<Width; k++){
            P_C[row*Width+col] += P_A[row*Width+k] * P_B[k*Width+col];
        }
    }
}

__global__ void  mat_mul(float *A, float *B, float *C, int width){
    __shared__ float As[TILE_WIDTH][TILE_WIDTH];
    __shared__ float Bs[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int row = by * TILE_WIDTH + ty;
    int col = bx * TILE_WIDTH + tx;

    float element_value = 0;

    for (int t=0; t < int (width+TILE_WIDTH - 1)/TILE_WIDTH; t ++){

        if ((row < width) && ((t*TILE_WIDTH + tx) < width)){
            As[ty][tx] = A[row*width + t*TILE_WIDTH + tx];
        }
        else{
            As[ty][tx] = 0.0f;
        }
    
        if ((col < width) && ((t*TILE_WIDTH + ty) < width)){
            Bs[ty][tx] = B[(t*TILE_WIDTH + ty)*width + col];
        }
        else{
            Bs[ty][tx] = 0.0f;
        }
        __syncthreads();

        
        for (int k=0; k < TILE_WIDTH; k++){
            element_value += As[ty][k]* Bs[k][tx];
        }
        __syncthreads();

    }
    if ((row < width) && (col < width)){
        C[row*width + col] = element_value;
    }

}


int main(){
    int rows = 2048;
    int cols = 2048;
    int size = rows * cols; 
    float *hA, *hB, *hC, *hC_naive; // Host arrays
    float *dA, *dB, *dC, *dC_naive; // Device arrays

    // Allocate memory on host
    hA = new float[size];
    hB = new float[size];
    hC = new float[size];
    hC_naive = new float[size];

    // Allocate memory on device
    cudaMalloc((void **)&dA, size * sizeof(float));
    cudaMalloc((void **)&dB, size * sizeof(float));
    cudaMalloc((void **)&dC, size * sizeof(float));

    cudaMalloc((void **)&dC_naive, size * sizeof(float));

    // Initialize host arrays (example values)
    for (int row = 0; row < rows; row ++){
        for (int col = 0; col < cols; col++){
            // hA[row*cols + col] = row*cols + col;
            hA[row*cols + col] = 1;
        }
    }

    // Initialize host arrays (example values)
    for (int row = 0; row < rows; row ++){
        for (int col = 0; col < cols; col++){
            // hB[row*cols + col] = row*cols + col;
            hB[row*cols + col] = 1;
        }
    }

    // Copy data from host to device
    cudaMemcpy(dA, hA, size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(dB, hB, size * sizeof(float), cudaMemcpyHostToDevice);


    // Kernel launch configuration
    dim3 blockSize(TILE_WIDTH, TILE_WIDTH, 1);
    dim3 gridSize((cols + TILE_WIDTH - 1) / TILE_WIDTH, (rows + TILE_WIDTH - 1) / TILE_WIDTH, 1);

    printf("%25s%25s%25s\n", "Exectution", "Bandwidth (GB/s)", "GPU Time (ms)");
    printf("\n");

    printf("%25s","Matrix Mul");

    cudaEvent_t start1, stop1;
    cudaEventCreate(&start1);
    cudaEventCreate(&stop1);
    float total_milliseconds1 = 0;
    int num_runs = 1;

    for (int i=0; i < num_runs; i++){
        cudaEventRecord(start1);
        mat_mul<<<gridSize, blockSize>>>(dA, dB, dC, rows);
        cudaEventRecord(stop1);
        cudaEventSynchronize(stop1);
        float milliseconds1 = 0;
        cudaEventElapsedTime(&milliseconds1, start1, stop1);
        total_milliseconds1 += milliseconds1;
    }
    float average_milliseconds1 = total_milliseconds1 / num_runs;
    cudaEventDestroy(start1);
    cudaEventDestroy(stop1);
    cudaMemcpy(hC, dC, size * sizeof(float), cudaMemcpyDeviceToHost);
    printf("%20.4f%25.4f\n", 2 * (size) * sizeof(float)* 1e-6 / average_milliseconds1, average_milliseconds1 );


    
    printf("%25s","Navie Matrix Mul");

    cudaEvent_t start2, stop2;
    cudaEventCreate(&start2);
    cudaEventCreate(&stop2);
    float total_milliseconds2 = 0;
    int num_runs1 = 1;

    for (int i=0; i < num_runs1; i++){
        cudaEventRecord(start2);
        navie_mat_mul<<<gridSize, blockSize>>>(dA, dB, dC_naive, rows);
        cudaEventRecord(stop2);
        cudaEventSynchronize(stop2);
        float milliseconds2 = 0;
        cudaEventElapsedTime(&milliseconds2, start2, stop2);
        total_milliseconds2 += milliseconds2;
    }
    float average_milliseconds2 = total_milliseconds2 / num_runs1;
    cudaEventDestroy(start2);
    cudaEventDestroy(stop2);
    cudaMemcpy(hC_naive, dC_naive, size * sizeof(float), cudaMemcpyDeviceToHost);

    printf("%20.4f%25.4f\n", 2 * size * sizeof(float)* 1e-6 / average_milliseconds2, average_milliseconds2 );

    
    printf("hA: \n ");
    for (int row = 0; row < 5; row ++){
        for (int col = 0; col < 5; col++){
            printf("%0.0f ", hA[row*cols + col]);
        }
        printf("\n");
    }
    
    printf("hB: \n");
    for (int row = 0; row < 5; row ++){
        for (int col = 0; col < 5; col++){
            printf("%0.0f ", hB[row*cols + col]);
        }
        printf("\n");
    }

    printf("\n\n");
    printf("hC: \n");
    for (int row = 0; row < 5; row ++){
        for (int col = 0; col < 5; col++){
            printf("%0.f ", hC[row*cols + col]);
        }
        printf("\n");
    }

    printf("\n\n");
    printf("hC_naive: \n");
    for (int row = 0; row < 5; row ++){
        for (int col = 0; col < 5; col++){
            printf("%0.f ", hC_naive[row*cols + col]/num_runs1);
        }
        printf("\n");
    }
    return 0;

}

In [ ]:
%%script bash
nvcc ./mul.cu -o t
nvprof ./t

# Convolution

In [50]:
%%writefile convolution.cu
#include <iostream>
#include <cuda_runtime.h>

#define filter_radius 2
#define tile_dim 32

__constant__ float hC[(2*filter_radius + 1)*(2*filter_radius + 1)];

__global__ void conv2d(float *inp, float *filt, float * oup, int rows, int cols){
    int gty = blockIdx.x * blockDim.x + threadIdx.x;
    int gtx = blockIdx.y * blockDim.y + threadIdx.y;

    float accumulate = 0;
    for (int fr = -1*filter_radius; fr < filter_radius+1; fr++){
        for (int fc = -1*filter_radius; fc < filter_radius+1; fc++){
            int gtx_new = gtx+fr;
            int gty_new = gty+fc;
            if (gtx_new >= 0 && gtx_new < rows && gty_new >= 0 && gty_new < cols){

                int abs_fr = fr + filter_radius;
                int abs_fc = fc + filter_radius;
                
                accumulate += filt[abs_fr * (2*filter_radius + 1) + abs_fc] * inp[gtx_new*cols + gty_new];
                
            }
        }
    oup[gtx*cols + gty] = accumulate;
    }
}


__global__ void conv2d_with_constant_memory(float *inp, float * oup, int rows, int cols){
    int gty = blockIdx.x * blockDim.x + threadIdx.x;
    int gtx = blockIdx.y * blockDim.y + threadIdx.y;

    float accumulate = 0;
    for (int fr = -1*filter_radius; fr < filter_radius+1; fr++){
        for (int fc = -1*filter_radius; fc < filter_radius+1; fc++){
            int gtx_new = gtx+fr;
            int gty_new = gty+fc;
            if (gtx_new >= 0 && gtx_new < rows && gty_new >= 0 && gty_new < cols){

                int abs_fr = fr + filter_radius;
                int abs_fc = fc + filter_radius;
                
                accumulate += hC[abs_fr * (2*filter_radius + 1) + abs_fc] * inp[gtx_new*cols + gty_new];
                
            }
        }
    oup[gtx*cols + gty] = accumulate;
    }
}


__global__ void conv2d_with_constant_memory_shared_memory(float *inp, float * oup, int rows, int cols){

    __shared__ int shared_tile[tile_dim][tile_dim+1];
    
    int gty = blockIdx.x * blockDim.x + threadIdx.x;
    int gtx = blockIdx.y * blockDim.y + threadIdx.y;

    shared_tile[threadIdx.y][threadIdx.x] = inp[gtx*rows + gty];
    __syncthreads();

    float accumulate = 0;
    for (int fr = -1*filter_radius; fr < filter_radius+1; fr++){
        for (int fc = -1*filter_radius; fc < filter_radius+1; fc++){
            // int gtx_new = gtx+fr;
            // int gty_new = gty+fc;
            // if (gtx_new >= 0 && gtx_new < rows && gty_new >= 0 && gty_new < cols){

            int abs_fr = fr + filter_radius;
            int abs_fc = fc + filter_radius;
            float tid_y_fc = threadIdx.y + fc;
            float tid_x_fr = threadIdx.x + fr;

            if (tid_y_fc >= 0 && tid_y_fc < tile_dim && tid_x_fr >= 0 && tid_x_fr < tile_dim ){
                accumulate += hC[abs_fr * (2*filter_radius + 1) + abs_fc] * shared_tile[threadIdx.y + fc][threadIdx.x + fr];
            }
                
        }
    oup[gtx*rows + gty] = accumulate;
    }
}

int main(){
    int rows = 1024;
    int cols = 1024;

    // int filter_radius = 2;

    float *hA, *hB, *hFilt;
    float *dA, *dB, *dC;

    hA = new float[rows*cols];
    hB = new float[rows*cols];
    hFilt = new float[(2*filter_radius + 1)*(2*filter_radius + 1)]; //filter

    cudaMalloc((void**)&dA,  rows*cols*sizeof(float));
    cudaMalloc((void**)&dB,  rows*cols*sizeof(float));
    cudaMalloc((void**)&dC,  rows*cols*sizeof(float));

    for (int i=0; i < rows; i++){
        for (int j=0; j < cols; j++){
            hA[i*cols + j] = 1;
        }
    }

    for (int i=0; i < 2*filter_radius + 1; i++){
        for (int j=0; j < 2*filter_radius + 1; j++){
            if (i == filter_radius || j == filter_radius){
                hFilt[i*(2*filter_radius + 1) + j] = 1;
            }
            else{
                hFilt[i*(2*filter_radius + 1) + j] = 0;
            }
        }
    }

    cudaMemcpy(dA, hA, rows * cols * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(dB, hB, rows * cols * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(dC, hFilt, (2*filter_radius + 1)*(2*filter_radius + 1) * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpyToSymbol(hC, hFilt, (2*filter_radius + 1)*(2*filter_radius + 1) * sizeof(float));


    dim3 block_size(tile_dim, tile_dim);
    dim3 grid_size((cols + block_size.x - 1)/block_size.x, (rows + block_size.y - 1)/block_size.y);
    int num_runs = 100;

    cudaEvent_t start1, stop1;
    cudaEventCreate(&start1);
    cudaEventCreate(&stop1);
    float total_ms1 = 0;

    printf("%25s%25s\n", "Exectution", "GPU Time (ms)");
    printf("\n");

    printf("%25s", "Naive Conv 2D");

    for (int i=0; i< num_runs; i++){
        cudaEventRecord(start1);
        conv2d<<<grid_size, block_size>>>(dA, dC, dB, rows, cols);
        cudaEventRecord(stop1);
        cudaEventSynchronize(stop1);
        float milliseconds1 = 0;
        cudaEventElapsedTime(&milliseconds1, start1, stop1);
        total_ms1 += milliseconds1;
    }
    float average_milliseconds1 = total_ms1 / num_runs;
    cudaEventDestroy(start1);
    cudaEventDestroy(stop1);
    cudaMemcpy(hB, dB, rows * cols * sizeof(float), cudaMemcpyDeviceToHost);
    printf("%20.4f", average_milliseconds1 );
    
    printf("\n");
    printf("%25s", "Constant Memory Conv 2D");
    cudaEvent_t start2, stop2;
    cudaEventCreate(&start2);
    cudaEventCreate(&stop2);
    float total_ms2 = 0;
    for (int i=0; i< num_runs; i++){
        cudaEventRecord(start2);
        conv2d_with_constant_memory<<<grid_size, block_size>>>(dA, dB, rows, cols);
        cudaEventRecord(stop2);
        cudaEventSynchronize(stop2);
        float milliseconds2 = 0;
        cudaEventElapsedTime(&milliseconds2, start2, stop2);
        total_ms2 += milliseconds2;
    }
    float average_milliseconds2 = total_ms2 / num_runs;
    cudaEventDestroy(start2);
    cudaEventDestroy(stop2);
    cudaMemcpy(hB, dB, rows * cols * sizeof(float), cudaMemcpyDeviceToHost);
    printf("%20.4f", average_milliseconds2 );


    printf("\n");
    printf("%25s", "Shared & Const Memory Conv 2D");
    cudaEvent_t start3, stop3;
    cudaEventCreate(&start3);
    cudaEventCreate(&stop3);
    float total_ms3 = 0;
    for (int i=0; i< num_runs; i++){
        cudaEventRecord(start3);
        conv2d_with_constant_memory_shared_memory<<<grid_size, block_size>>>(dA, dB, rows, cols);
        cudaEventRecord(stop3);
        cudaEventSynchronize(stop3);
        float milliseconds3 = 0;
        cudaEventElapsedTime(&milliseconds3, start3, stop3);
        total_ms3 += milliseconds3;
    }
    float average_milliseconds3 = total_ms3 / num_runs;
    cudaEventDestroy(start3);
    cudaEventDestroy(stop3);
    cudaMemcpy(hB, dB, rows * cols * sizeof(float), cudaMemcpyDeviceToHost);
    printf("%17.4f", average_milliseconds3 );



    // conv2d_with_constant_memory_shared_memory
    // printf("\nhB: \n");
    // for (int row = 0; row < 32; row ++){
    //     for (int col = 0; col < 32; col++){
    //         printf("%0.0f ", hB[row*cols + col]);
    //     }
    //     printf("\n");
    // }
    return 0;   
}

Overwriting convolution.cu


In [51]:
%%script bash
nvcc ./convolution.cu -o t1
nvprof ./t1

               Exectution            GPU Time (ms)

            Naive Conv 2D              0.2227
  Constant Memory Conv 2D              0.1699
Shared & Const Memory Conv 2D           0.1478

==949== NVPROF is profiling process 949, command: ./t1
==949== Profiling application: ./t1
==949== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   36.79%  21.696ms       100  216.96us  216.35us  217.50us  conv2d(float*, float*, float*, int, int)
                   27.83%  16.415ms       100  164.15us  163.01us  165.47us  conv2d_with_constant_memory(float*, float*, int, int)
                   24.06%  14.187ms       100  141.87us  138.53us  144.38us  conv2d_with_constant_memory_shared_memory(float*, float*, int, int)
                    7.00%  4.1282ms         3  1.3761ms  606.55us  2.9037ms  [CUDA memcpy DtoH]
                    4.32%  2.5481ms         4  637.03us     672ns  1.8039ms  [CUDA memcpy HtoD]
      API calls:   60.67%  105.24ms         3  35.080ms  111.77us  104.98ms  cudaMalloc
                   31.06%  53.881ms       300  179.60us  140.32us  230.36us  cudaEventSynchronize
                    6.24%  10

## Convolution
1. **Naive Convolution**
   -  a simple element-wise multiplication and summation within the filter window. 
   -  global memory access of the filter elements and input matrix
2. **Convolution with constant memory**
   -  here we are creating a constant memory, it is a special, read-only memory space on the GPU designed for storing data that remains constant during the execution of a kernel.
   -  used to store data that is read by many threads but does not change during the kernel's execution. This makes it good for storing coefficients, lookup tables, or other fixed parameters.
   -  Constant memory is cached on the GPU, which can significantly improve performance when all threads (or threads within a warp) access the same memory location.   
      Once initialized by the host (CPU), the GPU threads can only read from constant memory. There's a limited amount of constant memory available (typically 64KB).
   -  Still we are performing global memory access of the input element. This can be further optimized with shared memory where a tile in a block can save elements which can be read by all the           threads in a block.
3. **Shared and Constant Memory**
   -  Shared memory is used to reduce redundant global loads by reusing data, this reside on the GPU.
   -  Here the difference in execution is not significant in comparison to filter constant memory execution. one of the reason may be due to __syncthreads(); which holds the thread operations            until all threads in a block finishes exectuion until each thread in a block reaches __syncthreads(); there are other factors like tile/filter size, persistent memory bandwidth limits, and         implementation efficiency.

```
    Exectution            GPU Time (ms)
    
                Naive Conv 2D              0.2227
      Constant Memory Conv 2D              0.1699
    Shared & Const Memory Conv 2D           0.1478
    ==949== NVPROF is profiling process 949, command: ./t1
    ==949== Profiling application: ./t1
    ==949== Profiling result:
                Type  Time(%)      Time     Calls       Avg       Min       Max  Name
     GPU activities:   36.79%  21.696ms       100  216.96us  216.35us  217.50us  conv2d(float*, float*, float*, int, int)
                       27.83%  16.415ms       100  164.15us  163.01us  165.47us  conv2d_with_constant_memory(float*, float*, int, int)
                       24.06%  14.187ms       100  141.87us  138.53us  144.38us  conv2d_with_constant_memory_shared_memory(float*, float*, int, int)
                        7.00%  4.1282ms         3  1.3761ms  606.55us  2.9037ms  [CUDA memcpy DtoH]
                        4.32%  2.5481ms         4  637.03us     672ns  1.8039ms  [CUDA memcpy HtoD]
```


**Next Steps**
1. check for generalizing to rows != cols
2. check thread coarsening on performance
3. compute the memory bandwidth
4. reread PMPP convolution for other performance methods
5. remove the extra cells on the output matrix, here the size of input and output matrixs are same
